In [ ]:
%help

In [ ]:
%status

In [ ]:
%budget rinkeby 1

In [ ]:
%connect timeout=15m

In [ ]:
%pip install matplotlib pandas

In [ ]:
%upload life_expectancy.csv

In [ ]:
!ls -la workdir

In [ ]:
!lspci -vnn | grep VGA

In [ ]:
import torch
torch.cuda.get_device_name(0)

In [ ]:
import numpy as np
import pandas as pd
import random
import torch
from torch.autograd import Variable

training_df = pd.read_csv(filepath_or_buffer="workdir/life_expectancy.csv")

#training_df['median_house_value'] /= 1000.0

#TODO: take data from file (data.csv)

# create dummy data for training
# x_values = list(training_df[training_df.Country=="Poland"]['Year'])
x_values = [i for i in range(11)]
x_train = np.array(x_values, dtype=np.float32)
x_train = x_train.reshape(-1, 1)

# y_values = list(training_df[training_df.Country=="Poland"]['Life expectancy '])
y_values = [2*i + 1 + random.randint(-4,4) for i in x_values]
y_train = np.array(y_values, dtype=np.float32)
y_train = y_train.reshape(-1, 1)


class linearRegression(torch.nn.Module):
    def __init__(self, inputSize, outputSize):
        super(linearRegression, self).__init__()
        self.linear = torch.nn.Linear(inputSize, outputSize)

    def forward(self, x):
        out = self.linear(x)
        return out
    
inputDim = 1        # takes variable 'x' 
outputDim = 1       # takes variable 'y'
learningRate = 0.01 
epochs = 100

model = linearRegression(inputDim, outputDim)
##### For GPU #######
if torch.cuda.is_available():
    model.cuda()
    
criterion = torch.nn.MSELoss() 
optimizer = torch.optim.SGD(model.parameters(), lr=learningRate)

In [ ]:
for epoch in range(epochs):
    # Converting inputs and labels to Variable
    if torch.cuda.is_available():
        inputs = Variable(torch.from_numpy(x_train).cuda())
        labels = Variable(torch.from_numpy(y_train).cuda())
    else:
        inputs = Variable(torch.from_numpy(x_train))
        labels = Variable(torch.from_numpy(y_train))

    # Clear gradient buffers because we don't want any gradient from previous epoch to carry forward, dont want to cummulate gradients
    optimizer.zero_grad()

    # get output from the model, given the inputs
    outputs = model(inputs)

    # get loss for the predicted output
    loss = criterion(outputs, labels)
    print(loss)
    # get gradients w.r.t to parameters
    loss.backward()

    # update parameters
    optimizer.step()

    # print('epoch {}, loss {}'.format(epoch, loss.item()))

In [ ]:
from matplotlib import pyplot as plt

with torch.no_grad(): # we don't need gradients in the testing phase
    if torch.cuda.is_available():
        predicted = model(Variable(torch.from_numpy(x_train).cuda())).cpu().data.numpy()
    else:
        predicted = model(Variable(torch.from_numpy(x_train))).data.numpy()
    # print(predicted)

plt.clf()
plt.plot(x_train, y_train, 'go', label='True data', alpha=0.5)
plt.plot(x_train, predicted, '--', label='Predictions', alpha=0.5)
plt.legend(loc='best')
plt.savefig('workdir/plot1.png')
plt.show()

In [ ]:
from IPython.display import Image, display

display(Image('workdir/plot1.png'))

In [ ]:
%download plot1.png

In [ ]:
#TODO: show connection time
%disconnect